Our Model:  TensorFlow Simple MNIST ConvNet



In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [4]:
batch_size = 32
epochs = 32

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/32
1688/1688 [==============================] - 37s 3ms/step - loss: 0.2309 - accuracy: 0.9292 - val_loss: 0.0637 - val_accuracy: 0.9823
Epoch 2/32
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0883 - accuracy: 0.9731 - val_loss: 0.0447 - val_accuracy: 0.9860
Epoch 3/32
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0678 - accuracy: 0.9790 - val_loss: 0.0368 - val_accuracy: 0.9902
Epoch 4/32
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0579 - accuracy: 0.9819 - val_loss: 0.0359 - val_accuracy: 0.9895
Epoch 5/32
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0511 - accuracy: 0.9837 - val_loss: 0.0306 - val_accuracy: 0.9912
Epoch 6/32
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0442 - accuracy: 0.9860 - val_loss: 0.0299 - val_accuracy: 0.9922
Epoch 7/32
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0417 - accuracy: 0.9869 - val_loss: 0.0325 - val_accuracy

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02454003505408764
Test accuracy: 0.9932000041007996


Ok-- Now let's write out and then use the predict function.


In [13]:
from keras.models import load_model
model.save('mnist_model')
reconstructed_model = load_model('mnist_model')

INFO:tensorflow:Assets written to: mnist_model/assets


In [38]:
# Write to local
!zip -r /content/model.zip /content/mnist_model

  adding: content/mnist_model/ (stored 0%)
  adding: content/mnist_model/keras_metadata.pb (deflated 89%)
  adding: content/mnist_model/variables/ (stored 0%)
  adding: content/mnist_model/variables/variables.index (deflated 63%)
  adding: content/mnist_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/mnist_model/saved_model.pb (deflated 88%)
  adding: content/mnist_model/assets/ (stored 0%)


In [36]:
#import PIL as Pillow
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from imageio import imread


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':'1MkVgzJa8wBvyGCvmwhLQ9CrxlZaAnEfN'}) 
downloaded.GetContentFile('3.png') 

x = imread('3.png')
#compute a bit-wise inversion so black becomes white and vice versa
x = np.invert(x)
#make it the right size
x = np.resize(x,(28,28))
#convert to a 4D tensor to feed into our model
x = x.reshape(1,28,28,1)
x = x.astype('float32')
x /= 255

out = reconstructed_model.predict(test_input)
other = model.predict(test_input)
print(np.argmax(out))

2


In [26]:
print(out)
print(other)

[[5.5366887e-05 4.2487115e-14 9.9202877e-01 2.2459030e-06 1.3662069e-08
  7.6454153e-06 1.8368080e-06 1.7396359e-13 7.9041272e-03 2.0261018e-08]]
[[5.5366887e-05 4.2487115e-14 9.9202877e-01 2.2459030e-06 1.3662069e-08
  7.6454153e-06 1.8368080e-06 1.7396359e-13 7.9041272e-03 2.0261018e-08]]


Let the transfer learning begin with the TF ModelNetV2 as base.

---




In [ ]:
import tensorflow as tf
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(10)

In [ ]:
xtrain32 = tf.image.resize(
    x_train, [32,32], preserve_aspect_ratio=False,
    antialias=False, name=None
)

xtrain32 = tf.image.grayscale_to_rgb(
    xtrain32, name=None
)

xtest32 = tf.image.resize(
    x_test, [32,32], preserve_aspect_ratio=False,
    antialias=False, name=None
)

xtest32 = tf.image.grayscale_to_rgb(
    xtest32, name=None
)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(32,32,3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
base_model.summary()

In [ ]:
inputs = tf.keras.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [4]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

initial_epochs = 10
loss0, accuracy0 = model.evaluate(xtest32)

NameError: ignored

In [ ]:
history = model.fit(xtrain32, y_train,
                    epochs=50,
                    validation_split=0.1)

Epoch 1/50
1688/1688 [==============================] - 66s 39ms/step - loss: 0.3212 - accuracy: 0.1888 - val_loss: 0.3129 - val_accuracy: 0.2455
Epoch 2/50
1688/1688 [==============================] - 66s 39ms/step - loss: 0.3207 - accuracy: 0.1902 - val_loss: 0.3121 - val_accuracy: 0.2455
Epoch 3/50
1688/1688 [==============================] - 66s 39ms/step - loss: 0.3201 - accuracy: 0.1918 - val_loss: 0.3114 - val_accuracy: 0.2493
Epoch 4/50
1688/1688 [==============================] - 66s 39ms/step - loss: 0.3195 - accuracy: 0.1942 - val_loss: 0.3107 - val_accuracy: 0.2575
Epoch 5/50
1688/1688 [==============================] - 66s 39ms/step - loss: 0.3192 - accuracy: 0.1955 - val_loss: 0.3101 - val_accuracy: 0.2542
Epoch 6/50
1688/1688 [==============================] - 66s 39ms/step - loss: 0.3184 - accuracy: 0.1978 - val_loss: 0.3095 - val_accuracy: 0.2528
Epoch 7/50
1688/1688 [==============================] - 66s 39ms/step - loss: 0.3182 - accuracy: 0.1984 - val_loss: 0.3089 -

Now, let's try with a Keras MNIST pre-trained model.


In [9]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [15]:
from keras.models import load_model

downloaded = drive.CreateFile({'id':'1IbBbD6I56uvNdJXOyXSJmoXdiRrT1O7P'}) 
downloaded.GetContentFile('mnist.h5') 

pretrained_model = load_model('mnist.h5')
#out = model.predict(x)

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
score = pretrained_model.evaluate(x_test, y_test, verbose=0)

In [17]:
score

[0.029253464192152023, 0.9907000064849854]